In [189]:
from polars import DataFrame, col
from collections import namedtuple


In [190]:
Tx = namedtuple("Tx", ("ts", "price", "qty"))

market_price = 110
period_start_ts = 0
period_start_price = 101

txs = [
    Tx(1, 102, 2),
    Tx(3, 105, 5),
    Tx(4, 105.5, -6),
    Tx(5, 106.5, 3),
    Tx(7, 108, 1),
]

In [191]:
from polars import Float64, Int64


TXs = DataFrame(txs)
TXsb = TXs.filter(col("ts") <= period_start_ts)
TXsa = TXs.filter(col("ts") > period_start_ts)

TXsb = TXsb.select(
    col("ts").replace_strict({}, default=period_start_ts).cast(Int64),
    col("price").replace_strict({}, default=period_start_price).cast(Float64),
    col("qty").sum(),
).tail(1)

TXs = TXsb.extend(TXsa)
TXs

ts,price,qty
i64,f64,i64
1,102.0,2
3,105.0,5
4,105.5,-6
5,106.5,3
7,108.0,1


In [192]:
TXs = TXs.with_columns(
    ts_start=col("ts"),
    ts_end=col("ts").shift(-1),  # fill_null current_timestamp, or ts of market_price
    price_start=col("price"),
    price_end=col("price").shift(-1).fill_null(market_price),
    running_holding=col("qty").rolling_sum(len(TXs), min_samples=1),
    cost=col("qty") * col("price"),
).with_columns(
    holding_before=col("running_holding").shift(1).fill_null(0),
    growth_factor=col("price_end") / col("price_start"),
    running_value=col("running_holding") * col("price_end"),
    running_cost=col('cost').rolling_sum(len(TXs), min_samples=1)
)

TXs

ts,price,qty,ts_start,ts_end,price_start,price_end,running_holding,cost,holding_before,growth_factor,running_value,running_cost
i64,f64,i64,i64,i64,f64,f64,i64,f64,i64,f64,f64,f64
1,102.0,2,1,3,102.0,105.0,2,204.0,0,1.029412,210.0,204.0
3,105.0,5,3,4,105.0,105.5,7,525.0,2,1.004762,738.5,729.0
4,105.5,-6,4,5,105.5,106.5,1,-633.0,7,1.009479,106.5,96.0
5,106.5,3,5,7,106.5,108.0,4,319.5,1,1.014085,432.0,415.5
7,108.0,1,7,null,108.0,110.0,5,108.0,4,1.018519,550.0,523.5


In [193]:
twr = TXs.select(col("growth_factor").alias("twr")).product() - 1
val = TXs.select(col("running_value").alias("final_value")).tail(1)
cost = TXs.select(col("cost").sum().alias("final_cost"))
start_holding = TXs.head(1).select(
    (col("holding_before")*col("price_start")).alias("start_value")
)  # <-- this is probably wrong

twr = twr.to_dicts()[0]
val = val.to_dicts()[0]
cost = cost.to_dicts()[0]
start_holding = start_holding.to_dicts()[0]

print((twr | val | cost | start_holding))

twr, final_value, final_cost, start_value = (twr | val | cost | start_holding).values()
dollar_return = final_value - start_value - final_cost
mwr = dollar_return / final_cost


print(f"""
  {twr=}
  {mwr=}
  {start_value=}
  {final_value=}
  {final_cost=}
  {dollar_return=}
""")

{'twr': 0.07843137254901977, 'final_value': 550.0, 'final_cost': 523.5, 'start_value': 0.0}

  twr=0.07843137254901977
  mwr=0.050620821394460364
  start_value=0.0
  final_value=550.0
  final_cost=523.5
  dollar_return=26.5



In [194]:
import altair as alt

values = TXs.to_dicts()
chart = alt.Chart(alt.Data(values=values)).transform_fold(
    ['running_cost', 'running_value'],
    as_=['series', 'value']
).mark_area(opacity=0.6).encode(
    x='ts:T',
    y=alt.Y('value:Q', stack=None),   # disable stacking to overlay
    color='series:N'
)
chart

alt.Chart(...)